In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np

from ICARUS.Database.Database_3D import Database_3D

In [ ]:
import os
HOMEDIR = os.getcwd()

# Open Database

In [ ]:
db = Database_3D(HOMEDIR)

### Select Plane

In [ ]:
planenames = db.getPlanes()
for i, planename in enumerate(planenames):
    print(f"{i} : `{planename}")
# index = int(input("Select a plane: "))

plane = db.Planes[planenames[0]]
planenames = planenames[0:1]
# dynplane = db.dynPlanes[planenames[index]]

# Add XFLR Polars

In [ ]:
from ICARUS.Software.XFLR5.polars import readPolars3D
from ICARUS.Database import XFLRDB
for name in planenames:
    if name.startswith("XFLR"):
        continue
    try:
        XFLR5PLANEDIR = os.path.join(XFLRDB,f"{name}.txt")
        readPolars3D(db,XFLR5PLANEDIR, name)
        print(f"Imported XFLR polar for {name}")
        planenames.append(f"XFLR_{name}")
    except FileNotFoundError:
        print(f"No XFLR polar found for {name}")
        print(f"Importing XFLR polar for Hermes")
        if "XFLR_Hermes" not in planenames:
            XFLR5PLANEDIR = os.path.join(XFLRDB,f"Hermes.txt")
            readPolars3D(db,XFLR5PLANEDIR, 'Hermes')
            planenames.append(f"XFLR_Hermes")

# Airplane Visualization

In [ ]:
plane.visAirplane()

# Polars

In [ ]:
from ICARUS.Visualization.airplanePolars import plotAirplanePolars

In [ ]:
db.makeData()
plotAirplanePolars(db.Data,planenames,["2D",'Potential'],size=(16,7))

# Case Convergence

In [ ]:
from ICARUS.Visualization.convergence import plotConvergence

In [ ]:
angles2show = np.arange(-1,3) # All
plotConvergence(db.Convergence, plane.name, angles = angles2show,
                solvers= ["2D"],plotError = False,size=(16,7))

# Sensitivity Plot

In [ ]:
from ICARUS.Visualization.sensitivity import plotSensitivity

In [ ]:
# pertr = dynplane.pertubResults
# print(dynplane.trim)
# trim = pertr[pertr['Type'] == 'Trim']
# dynplane.sensResults['u'].sort_values(by=['Epsilon'])
# plotSensitivity(dynplane.sensResults,dynplane,trim,relative = 0,vars2s=['r'])

# Wake Visualization

In [ ]:
from ICARUS.Visualization.GNVPwake import GNVPwake
from ICARUS.Database.Database_3D  import ang2case

In [ ]:
CASE = ang2case(1.)
GNVPwake(db.Planes["Hermes"],CASE)

# Loads Visualization

In [ ]:
from ICARUS.Visualization.GNVPstrips import GNVPstrips3D,GNVPstrips2D
plane = db.Planes['Hermes']
NBs = [i+1 for i in range(len(plane.surfaces))]
print(plane)
print("NBs to Visualize: ",NBs)

stripData = GNVPstrips2D(plane,CASE, NBs[2], category= 'Wind')

# Dynamics

In [ ]:
dyn = db.dynPlanes['Hermes_MRes']

In [ ]:
dyn.stabilityFD()
print(dyn.SBderivativesDS)
#   Longitudinal derivatives
#   Xu=    -0.16379         Cxu=   -0.049477
#   Xw=      1.1494         Cxa=     0.34723
#   Zu=     -3.0117         Czu=  -0.0035415
#   Zw=     -19.528         CLa=      5.8991
#   Zq=     -1.8321         CLq=      9.1645
#   Mu= -0.00060463         Cmu=  -0.0015122
#   Mw=    -0.45664         Cma=     -1.1421
#   Mq=    -0.39098         Cmq=     -16.193

#   Lateral derivatives
#   Yv=     -0.5496         CYb=    -0.16602
#   Yp=   -0.017022         CYp=  -0.0045504
#   Yr=     0.32383         CYr=    0.086568
#   Lv=    0.039731         Clb=   0.0053106
#   Lp=     -5.6227         Clp=     -0.6651
#   Lr=      1.0377         Clr=     0.12275
#   Nv=     0.28879         Cnb=    0.038602
#   Np=    -0.57586         Cnp=   -0.068117
#   Nr=     -0.1259         Cnr=   -0.014893


In [ ]:
from ICARUS.Flight_Dynamics.Stability.state import State

a = State(dyn, 'Trim', dyn.trim)
a.plotEig()
